In [7]:
import selectors
import socket
import pickle, struct

In [12]:
def send_msg(sock: socket, data):
    serialized_data = pickle.dumps(data)
    sock.sendall(struct.pack('>I', len(serialized_data)) + serialized_data)


def recv_msg(sock : socket):
    # Read message length and unpack it into an integer
    raw_msglen = sock.recv(4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # Read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # Helper function to recv n bytes or return None if EOF is hit
    data = bytearray()
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data.extend(packet)
    return pickle.loads(data)

In [10]:
HOST = ""  # Standard loopback interface address (localhost)
PORT = 65432  # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        while True:
            msg = recv_msg(conn)
            print(msg)
            if not msg:
                break
            send_msg(conn, msg)

Connected by ('127.0.0.1', 50648)
{2: {'sock': 'sock', 'addr': 'myaddr@enst.fr'}}
{2: {'sock': 'sock', 'addr': 'myaddr@enst.fr'}}
None


In [11]:
HOST = ""  # Standard loopback interface address (localhost)
PORT = 65432  # Port to listen on (non-privileged ports are > 1023)

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen()
    conn, addr = s.accept()
    with conn:
        print(f"Connected by {addr}")
        msg = recv_msg(conn)
        print(msg)
        msg = recv_msg(conn)
        print(msg)

Connected by ('127.0.0.1', 50664)
{2: {'sock': 'sock', 'addr': 'myaddr@enst.fr'}}
{2: {'sock': 'sock', 'addr': 'myaddr@enst.fr'}}


In [3]:
sel = selectors.DefaultSelector()

In [4]:
host, port = "", 65432

lsock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
lsock.bind((host, port))
lsock.listen()
print(f"Listening on {(host, port)}")
lsock.setblocking(False)
sel.register(lsock, selectors.EVENT_READ | selectors.EVENT_WRITE, data=['None', 'test'])

Listening on ('', 65432)


SelectorKey(fileobj=<socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 65432)>, fd=68, events=3, data=['None', 'test'])

In [5]:
events = sel.select(timeout=None)

In [6]:
events

[(SelectorKey(fileobj=<socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('0.0.0.0', 65432)>, fd=68, events=3, data=['None', 'test']),
  1)]

In [7]:
for key, mask in events:
    data = key.data
    for d in data:
        print(d)

None
test


In [25]:
print(sel.get_map())

KeyError: '0 is not registered'

In [ ]:
print(sel.get_map())

KeyError: '0 is not registered'